# How to use st_hsdatalog HSD_link package - \[Communication Features\]
---
<br>
<span style="color:cyan; font-weight:bold">[!!!] Connect an HSDatalog (v1 or v2) compatible device, then import the HSD_link module. [!!!]</span>
<p>Using this module with a connected device you will be able to:</p>
<ul>
<li>retrieve configuration details (both capabilities and current setting)</li>
<li>update device sensors parameters</li>
<li>control an acquisition saving sensor data transfered</li>
<li>annotate real-time acquisitions</li>
</ul>

In [ ]:
from st_hsdatalog.HSD_link.HSDLink import HSDLink

Initialize the HSD_link module with a specific device command set. 'stwin_hsd' string is used to create the right CommandManager internally.

In [ ]:
from st_hsdatalog.HSD_link.HSDLink_v2 import HSDLink_v2

hsd_link_factory = HSDLink()
hsd_link = hsd_link_factory.create_hsd_link()

hsd_2 = isinstance(hsd_link, HSDLink_v2)

"stwin_hsd" USB communication is based on a DLL which guarantees stable and robust communication at very high speeds.
<p>To get DLL version:</p>

In [ ]:
print(HSDLink.get_version(hsd_link))

<p>This communication framework allow you to interact with a connected device using commands that could be grouped by several different types:</p>
<h2>GET commands</h2>
<hr>
<p>You can obtain easily the list of all connected Devices</p>

In [ ]:
dev_list = HSDLink.get_devices(hsd_link)
for dev in dev_list:
    print(dev)

Or select a specific one

In [ ]:
device_id = 0

device = HSDLink.get_device(hsd_link, device_id)
print(device)

device_alias = HSDLink.get_device_alias(hsd_link, device_id)
print(device_alias)

Create a new HSDatalog to store the device model obtained from the physical board.
<p><i>* All operations in this notebook can also be performed without using this <b style="color:cyan">[HSDatalog]</b> object, which however makes it easier to perform some tasks</i></p>

In [ ]:
from st_hsdatalog.HSD.HSDatalog import HSDatalog
hsd_factory = HSDatalog()
hsd= hsd_factory.create_hsd(device_config = device)

HSDatalog.set_device(hsd, device)

A list of high-level device information could be retrieved from the device model obtained
<p style="font-style: italic">the same thing can be done also using the <b style="color:cyan">[HSDatalog]</b> object </p>

In [ ]:
device_info = HSDLink.get_device_info(hsd_link, device_id)
HSDatalog.present_device_info(hsd, device_info)
device_info = HSDatalog.get_device_info(hsd)
HSDatalog.present_device_info(hsd, device_info)

The list of sensors configuration can be obtained directly from the connected device and filtered by two parameters: the activation status and the sensor type.
<p><li> <u>Only active sensors</u> (<b style="color:cyan">[HSDatalog]</b> utility function is used to present the sensor list): </p>

In [ ]:
sensor_list = HSDLink.get_sensor_list(hsd_link, device_id, only_active=True)
HSDatalog.present_sensor_list(hsd, sensor_list)

<p><li><u>All sensors</u>, even those not active (<b style="color:cyan">[HSDatalog]</b> utility function is used to present the sensor list): </p>

In [ ]:
sensor_list = HSDLink.get_sensor_list(hsd_link, device_id, only_active=False)
HSDatalog.present_sensor_list(hsd, sensor_list)

<p><li><u>Active sensors filtered by a specific type</u> (e.g. MIC) (<b style="color:cyan">[HSDatalog]</b> utility function is used to present the sensor list): </p>

In [ ]:
sensor_list = HSDLink.get_sensor_list(hsd_link, device_id, only_active = True, type_filter="MIC")
HSDatalog.present_sensor_list(hsd, sensor_list)

<p>Sensors configuration list can also be extracted from the previously retrieved Device object using the <b style="color:cyan">[HSDatalog]</b> object</p>

In [ ]:
sensor_list = HSDatalog.get_sensor_list(hsd, type_filter="ACC")
HSDatalog.present_sensor_list(hsd, sensor_list)


<p>It is also possible to directly select a specific sensor from the device model using the <b style="color:cyan">[HSDatalog]</b> object. For each sensor, it is possible to obtain descriptor and status parameters</p>

In [ ]:
if hsd_2:
    sensor_name = "ism330dhcx_acc"
else:
    sensor_name = "ISM330DHCX"
    
sensor = HSDatalog.get_sensor(hsd, sensor_name)
HSDatalog.present_sensor(hsd, sensor)

And from a specific sub sensor to extract specific parameters:

In [ ]:
if hsd_2:
    sensor_name = "iis2dlpc_acc"
    s_id = None
    ss_id = None
else:
    sensor_name = None
    s_id = 0
    ss_id = 0

sensor_enabled = HSDLink.get_sensor_enabled(hsd_link, device_id, sensor_name, s_id, ss_id)
sensor_odr = HSDLink.get_sensor_odr(hsd_link, device_id, sensor_name, s_id, ss_id)
sensor_fs = HSDLink.get_sensor_fs(hsd_link, device_id, sensor_name, s_id, ss_id)
sensor_spts = HSDLink.get_sensor_spts(hsd_link, device_id, sensor_name, s_id, ss_id)
print("Enabled: {}".format(sensor_enabled))
print("ODR (Property Enum value in HSDatalog2): {}".format(sensor_odr))
print("FS (Property Enum value in HSDatalog2): {}".format(sensor_fs))
print("SamplePerTimestamp: {}".format(sensor_spts))

<span style="color:red; font-weight: bold">[HSDatalog2]</span> <p>In Datalog2 some device Properties could be defined as Enumerative. Their values are defined in the related DTDL Property.</p><p>A way to get the numerical value for the related Property is described in the following code cell.</p></span>

In [ ]:
if hsd_2:
    device = HSDLink.get_device(hsd_link, device_id)
    HSDatalog.set_device(hsd, device)
    sensor = HSDatalog.get_sensor(hsd, sensor_name)
    print("ODR: {}".format(HSDatalog.get_sensor_odr(hsd, sensor)))
    print("FS: {}".format(HSDatalog.get_sensor_fs(hsd, sensor)))

<p>In addiction you can get other information about available Tag classes in FW:</p>
<li><u>SW Tag</u> classes

In [ ]:
sw_tag_classes = HSDLink.get_sw_tag_classes(hsd_link, device_id)
HSDatalog.present_sw_tag_classes(hsd, sw_tag_classes)
if hsd_2: #Additional APIs
    print(hsd_link.get_sw_tag_class_label_by_id(device_id,tag_class_id=0))
    print(hsd_link.get_sw_tag_class_enabled_by_id(device_id,tag_class_id=0))
    print(hsd_link.get_sw_tag_class_status_by_id(device_id,tag_class_id=0))
    print(hsd_link.get_sw_tag_class_label(device_id,comp_name="sw_tag0"))
    print(hsd_link.get_sw_tag_class_enabled(device_id,comp_name="sw_tag0"))
    print(hsd_link.get_sw_tag_class_status(device_id,comp_name="sw_tag0"))

<p><li><u>HW Tag</u> classes</p>

In [ ]:
hw_tag_classes = HSDLink.get_hw_tag_classes(hsd_link, device_id)
if hw_tag_classes is not None:
    HSDatalog.present_hw_tag_classes(hsd, hw_tag_classes)
else:
    print("No HW Tag classes available")

<li><u>Maximum number of annotations</u> supported per single acquisition</li>

In [ ]:
max_tag_per_acq = HSDLink.get_max_tags_per_acq(hsd_link, device_id)
print("Max tags per acquisition: {}".format(max_tag_per_acq))

<h2>SET Commands
<hr>

SET single sensor parameter:

In [ ]:
if hsd_2:
    sensor_name = "ism330dhcx_acc"
    sensor_type = None
    print("--> SET enable Property = True")
    hsd_link.set_sensor_enable(d_id = 0, new_status=True, comp_name=sensor_name)
    print("Enable: {}".format(hsd_link.get_sensor_enable(d_id = 0, sensor_name=sensor_name)))
    print("--> SET odr Property = 1666.0 Hz (DTDL enum=7)")
    hsd_link.set_sensor_odr(d_id = 0, new_odr=7, comp_name=sensor_name)
    print("ODR: {}".format(hsd_link.get_sensor_odr(d_id = 0, sensor_name=sensor_name)))
    print("--> SET fs Property = 8 g (DTDL enum=2)")
    hsd_link.set_sensor_fs(d_id = 0, new_fs=2, comp_name=sensor_name)
    print("FS: {}".format(hsd_link.get_sensor_fs(d_id = 0, sensor_name=sensor_name)))
    print("--> SET SamplePerTimestamp = 500")
    hsd_link.set_sensor_samples_per_ts(d_id = 0, new_spts=500, comp_name=sensor_name)
    print("SamplePerTimestamp: {}".format(hsd_link.get_sensor_samples_per_ts(d_id = 0, sensor_name=sensor_name)))
else:
    print("SensorName: {}".format(hsd_link.get_sensor_name(d_id = 0, s_id = 5)))
    print("SubSensorType: {}".format(hsd_link.get_sub_sensor_type(d_id = 0, s_id = 5, ss_id = 0)))
    print("--> SET isActive = True")
    hsd_link.set_sub_sensor_active(d_id = 0, s_id = 5, ss_id = 0, new_status = True)
    print("isActive: {}".format(hsd_link.get_sub_sensor_isActive(d_id = 0, s_id = 5, ss_id = 0)))
    print("--> SET ODR = 3332.0")
    hsd_link.set_sub_sensor_odr(d_id = 0, s_id = 5, ss_id = 0, odr_value = 3332.0)
    print("ODR: {}".format(hsd_link.get_sub_sensor_odr(d_id = 0, s_id = 5, ss_id = 0)))
    print("--> SET FS = 4.0")
    hsd_link.set_sub_sensor_fs(d_id = 0, s_id = 5, ss_id = 0, fs_value = 4.0)
    print("FS: {}".format(hsd_link.get_sub_sensor_fs(d_id = 0, s_id = 5, ss_id = 0)))
    print("--> SET SamplePerTimestamp = 1000")
    hsd_link.set_samples_per_timestamp(d_id = 0, s_id = 5, ss_id = 0, spts_value=1000)
    print("SamplePerTimestamp: {}".format(hsd_link.get_sub_sensor_sample_per_ts(d_id = 0, s_id = 5, ss_id = 0)))

Upload an ucf file to configure the MLC sensor on the board (also if it isn't enabled)

In [ ]:
if hsd_2:
    ucf_file_path = "..\\..\\STWIN.box_config_examples\\UCF_examples\\ism330dhcx_six_d_position.ucf"
else:
    ucf_file_path = "..\\..\\STWIN_config_examples\\UCF_examples\\ism330dhcx_six_d_position.ucf"
HSDLink.upload_mlc_ucf_file(hsd_link, device_id, ucf_file_path)

Device configuration update from JSON file

In [ ]:
if hsd_2:
    json_file_path = "..\\..\\STWIN.box_config_examples\\env_sensors.json"
else:
    json_file_path = "..\\..\\STWIN_config_examples\\EnvSensors.json"

HSDLink.update_device(hsd_link, device_id, json_file_path)
sensor_list = HSDLink.get_sensor_list(hsd_link, device_id, only_active=False)
HSDatalog.present_sensor_list(hsd, sensor_list)

Set next acquisition name and description

In [ ]:
HSDLink.set_acquisition_info(hsd_link, device_id, "acq_name_00","acq_notes_00")
if hsd_2: #HSDatalog2 Additional APIs
    hsd_link.set_acquisition_name(0,"acq_name_00")
    hsd_link.set_acquisition_description(0,"acq_notes_00")
print(HSDLink.get_acquisition_info(hsd_link, device_id))

[HSDatalog1 Only] Configure HW Tags (label and activation status)

In [ ]:
hw_tag_classes = HSDLink.get_hw_tag_classes(hsd_link, device_id)
if hw_tag_classes is not None:
    print(hw_tag_classes)
    hsd_link.set_hw_tag_disabled(d_id = 0 , t_id = 0)
    hsd_link.set_hw_tag_label(d_id = 0 , t_id = 0 , label= "Running")
    hw_tag_classes = hsd_link.get_hw_tag_classes(d_id = 0)
    print(hw_tag_classes)
else:
    print("No HW Tag classes available")

Configure SW Tags labels

In [ ]:
sw_tag_classes = HSDLink.get_sw_tag_classes(hsd_link, device_id)
print(sw_tag_classes)

if hsd_2:
    hsd_link.set_sw_tag_class_enabled_by_id(d_id = 0, tag_class_id = 3, new_status = False)
    hsd_link.set_sw_tag_class_label_by_id(d_id = 0, tag_class_id = 3, new_label = "Walking")
    hsd_link.set_sw_tag_class_enabled(d_id = 0, comp_name="sw_tag4", new_status = False)
    hsd_link.set_sw_tag_class_label(d_id = 0, comp_name="sw_tag4", new_label = "Walking")
else:
    hsd_link.set_sw_tag_label(d_id = 0 , t_id = 3 , label= "Walking")

sw_tag_classes = HSDLink.get_sw_tag_classes(hsd_link, device_id)
print(sw_tag_classes)

<h2>LOG CONTROL Commands </h2>
<hr>

<span style="color:red; font-weight: bold">[HSDatalog2 Only]</span> Set RTC Time. Send to the board the current datetime formatted as follow:
<ul><b>"%Y%m%d_%H_%M_%S"</b></ul>

In [ ]:
if hsd_2:
    hsd_link.set_rtc_time(device_id)

Once the configuration phase is concluded, it is possible to start the acquisition process.
<p>With an acquisition in progress, you can get sensor data with for each active sensors.</p>
<p><i>* In the following three code cells we start an acquisition retrieving data from all active sensors and save them in the acquisition folder. A dedicated thread will be created for each active sensor</i></p>

In [ ]:

import time

threads_stop_flags = []
sensor_data_files = []

sensor_list = HSDLink.get_sensor_list(hsd_link, device_id, only_active=True)
HSDatalog.present_sensor_list(hsd, sensor_list)

HSDLink.start_log(hsd_link, device_id)

for s in sensor_list:
    HSDLink.start_sensor_acquisition_thread(hsd_link, device_id, s, threads_stop_flags, sensor_data_files, print_data_cnt = True)
    
time.sleep(3)

Tag an acquisition in progress. (wait for 3 seconds, then 3 seconds of SW Tag Class 0 ->(label defined before as "Walking"), followed by 3 seconds of SW Tag Class 1.

In [ ]:

time.sleep(3)

tag_id = 0
HSDLink.set_sw_tag_on(hsd_link, device_id, tag_id=tag_id)
time.sleep(3)
HSDLink.set_sw_tag_off(hsd_link, device_id, tag_id=tag_id)

if hsd_2: #HSDatalog2 Additional APIs
    hsd_link.set_sw_tag_on(0,"sw_tag1")
    time.sleep(3)
    hsd_link.set_sw_tag_off(0,"sw_tag1")
else:
    tag_id = 1
    HSDLink.set_sw_tag_on(hsd_link, device_id, tag_id)
    time.sleep(3)
    HSDLink.set_sw_tag_off(hsd_link, device_id, tag_id)
    
time.sleep(3)

<p>To stop the process and save all necessary acquisition files in the right folder, you have to launch the following commands.</p>
<p>In addition to sensor data files(.dat), the following JSON files will also be saved in the acquisitino folder:</p>
<p>HSDatalog_v1</p>
<li>Device configuration file <b>[DeviceConfig.json]</b> containing a complete device capability description and the current device status snapshot</li>
<li>Acquisition info file <b>[AcquisitionInfo.json]</b> containing a list of all annotations taken during the acquisition process</li>
<p>HSDatalog_v2</p>
<li>Device configuration file <b>[device_config.json]</b> containing the current device status snapshot (Properties values)</li>
<li>Acquisition info file <b>[acquisition_info.json]</b> containing a list of all annotations taken during the acquisition process (Acquisition Info Component Status)</li>
<br>
<i>* Each of these files contains an identification string to link them together and with a specific acquisition</i>

In [ ]:
for sf in threads_stop_flags:
    sf.set()
for f in sensor_data_files:
    f.close()

HSDLink.stop_log(hsd_link, device_id)
HSDLink.save_json_device_file(hsd_link, device_id)
HSDLink.save_json_acq_info_file(hsd_link, device_id)